Celem ćwiczenia jest prezentacja możliwości wykorzystania metod uczenia maszynowego
w rozpoznawaniu komend głosowych.

Materiałem użytym w ćwiczeniu będą próbki nagrań komend z poprzednich ćwiczeń.
Przypomnijmy, w zbiorze jest:
- 30 (+szum) komend,
- nagranych ok. 1,7-2,4 tys. razy.

Treścią zadania jest klasyfikacja komend:
- klasa (class) = komenda,
- próbka (sample) = nagranie.

Ćwiczenie podzielone jest na cztery etapy:
1. Ekstrakcja cech z próbek
2. Podział próbek na treningowe i testowe
3. Uczenie klasyfikatora
4. Klasyfikacja nieznanej próbki
5. Ocena działania klasyfikatora


In [1]:
# import bibliotek
import os
import librosa
import pickle
import sklearn
import numpy as np
import random
from sklearn.model_selection import train_test_split


In [2]:
# 1. Ekstrakcja cech z próbek
# Lista ekstraktorów cech: https://librosa.github.io/librosa/feature.html

# ścieżka z folderami z nagraniami - próbki:
train_audio_path = 'C:/Users/Aayli/Documents/Delete'
commands = os.listdir(train_audio_path);  

# użyjemy MFCC, tj. Mel-frequency cepstral coefficients
# jednym z parametrów jest liczba wyjściowych współczynników:
n_mfcc = 9


# wczytujemy każde nagranie, wykonujemy ekstrakcję i zapamiętujemy jej wynik
classes = []
class_size = 100 # użyjemy tylko po kilka danych
samples = []
labels = []
print("\nClass: ", end = '')
for command in commands:
    if command == 'testing_list.txt' or command == 'validation_list.txt' or command == '_background_noise_' or command == 'tmp':
        continue
    classes.append(command)
    print(command, end = ', ')
    sample_num = 0
    for file in os.listdir(train_audio_path + '/' + command + '/'):
        # print(".", end = '')
        sample_num += 1
        # przykład z MFCC:
        # y - waveform, sr - sampling rate, n_mfcc - number of MFCCs to return
        y, sr = librosa.load(train_audio_path + '/' + command + '/' + file)
        mfcc_seq = librosa.feature.mfcc(y = y, sr = sr, n_mfcc = n_mfcc)
        mean_mfcc_seq = []
        for feature in mfcc_seq:
            mean_mfcc_seq.append(np.mean(feature))
        samples.append(mean_mfcc_seq)
        labels.append(command)
        if sample_num == class_size:
            break

# warto podglądnąć, dla mniejszych n_mfcc, np. 5 co kryje mfcc_seq i mean_mfcc_seq

# zapiszmy bazę do późniejszej pracy
database = open('C:/Users/Aayli/Documents/Delete/tmp/database.pkl', 'wb')
pickle.dump([classes, samples, labels], database)
database.close()



Class: bed, bird, cat, dog, down, eight, five, four, go, happy, house, left, marvin, nine, no, off, on, one, right, seven, sheila, six, stop, three, tree, two, up, wow, yes, zero, 

In [3]:
# 2. Wybór danych treningowych i testowych

# Zautomatyzowany i losowy sposób to użycie:
# "from sklearn.model_selection import train_test_split" oraz funkcji "train_test_split()"

# wczytanie danych
database = open('C:/Users/Aayli/Documents/Delete/tmp/database.pkl', 'rb')
classes, samples, labels = pickle.load(database)
database.close()

# wybierzmy 10 pierwszych próbek z każdej klasy jako uczące
# oraz kolejne 2 z każdej klasy jako testowe
# uwaga! train_size + test_size NIE może przekroczyć class_size (poprzedni punkt)
train_size = 50
test_size = 10
train_samples = []
train_labels = []
test_samples = []
test_labels = []

available_indexes = np.array(range(class_size))
random.shuffle(available_indexes)
for classname in classes:
    train_size_index = 0
    test_size_index = 0
    for i in available_indexes:
        if labels[i] == classname and train_size_index < train_size:
            train_samples.append(samples[i])
            train_labels.append(labels[i])
            train_size_index += 1
        elif labels[i] == classname and test_size_index < test_size:
            test_samples.append(samples[i])
            test_labels.append(labels[i])
            test_size_index += 1
        if train_size_index == train_size and test_size_index == test_size:
            break

# proszę sprawdzić, czy dane zostały wybrane prawidłowo
print(np.size(samples, 0))
print(np.size(train_samples, 0))
print(np.size(train_labels, 0))
print(np.size(test_samples, 0))
print(np.size(test_labels, 0))
    
    


3000
50
50
10
10


In [4]:
train_samples, test_samples, train_labels, test_labels = train_test_split(samples, labels, test_size=0.3333, random_state=42)

# proszę sprawdzić, czy dane zostały wybrane prawidłowo
print(np.size(samples, 0))
print(np.size(train_samples, 0))
print(np.size(train_labels, 0))
print(np.size(test_samples, 0))
print(np.size(test_labels, 0))

3000
2000
2000
1000
1000


In [5]:
# 3. Uczenie klasyfikatora danymi treningowymi

# Lista klasyfikatorów: https://stackabuse.com/overview-of-classification-methods-in-python-with-scikit-learn/
# Prosty tutorial: https://www.digitalocean.com/community/tutorials/how-to-build-a-machine-learning-classifier-in-python-with-scikit-learn

# skalowanie/normalizacja danych
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(train_samples)
train_samples = scaler.transform(train_samples)
test_samples = scaler.transform(test_samples)

# uczenie
from sklearn.neighbors import KNeighborsClassifier
k = 3
classifier = KNeighborsClassifier(n_neighbors = k)
classifier.fit(train_samples, train_labels)


# 4. Klasyfikacja nieznanych próbek
predicted_labels = classifier.predict(test_samples)

# zobaczmy wynik - zakomentować dla dużych zbiorów
# print(test_labels)
# print(predicted_labels)


# 5. Ocena klasyfikatora
# print(test_labels == predicted_labels)
quality = np.sum(test_labels == predicted_labels) / len(test_labels) * 100;
print('MFCC: Ocena klasyfikatora: %.2f %%' % quality)








MFCC: Ocena klasyfikatora: 23.80 %


In [17]:
# 1. Ekstrakcja cech z próbek
# Lista ekstraktorów cech: https://librosa.github.io/librosa/feature.html

# ścieżka z folderami z nagraniami - próbki:
train_audio_path = 'C:/Users/Aayli/Documents/Delete'
commands = os.listdir(train_audio_path);  


# wczytujemy każde nagranie, wykonujemy ekstrakcję i zapamiętujemy jej wynik
classes = []
class_size = 100 # użyjemy tylko po kilka danych
samples = []
labels = []
for command in commands:
    if command == 'testing_list.txt' or command == 'validation_list.txt' or command == '_background_noise_' or command == 'tmp':
        continue
    classes.append(command)
    sample_num = 0
    for file in os.listdir(train_audio_path + '/' + command + '/'):
        sample_num += 1
        y, sr = librosa.load(train_audio_path + '/' + command + '/' + file)
        mfcc_seq = librosa.feature.spectral_rolloff(y=y, sr=sr)
        mean_mfcc_seq = []
        for feature in mfcc_seq:
            mean_mfcc_seq.append(np.mean(feature))
        samples.append(mean_mfcc_seq)
        labels.append(command)
        if sample_num == class_size:
            break

# warto podglądnąć, dla mniejszych n_mfcc, np. 5 co kryje mfcc_seq i mean_mfcc_seq

# zapiszmy bazę do późniejszej pracy
database = open('C:/Users/Aayli/Documents/Delete/tmp/database.pkl', 'wb')
pickle.dump([classes, samples, labels], database)
database.close()
# 2. Wybór danych treningowych i testowych

# Zautomatyzowany i losowy sposób to użycie:

# wczytanie danych
database = open('C:/Users/Aayli/Documents/Delete/tmp/database.pkl', 'rb')
classes, samples, labels = pickle.load(database)
database.close()

# wybierzmy 10 pierwszych próbek z każdej klasy jako uczące
# oraz kolejne 2 z każdej klasy jako testowe
# uwaga! train_size + test_size NIE może przekroczyć class_size (poprzedni punkt)
train_size = 12
test_size = 10

train_samples = []
train_labels = []
test_samples = []
test_labels = []
train_samples, test_samples, train_labels, test_labels = train_test_split(samples, labels, test_size=0.33, random_state=42)

# 3. Uczenie klasyfikatora danymi treningowymi

# Lista klasyfikatorów: https://stackabuse.com/overview-of-classification-methods-in-python-with-scikit-learn/
# Prosty tutorial: https://www.digitalocean.com/community/tutorials/how-to-build-a-machine-learning-classifier-in-python-with-scikit-learn

# skalowanie/normalizacja danych
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(train_samples)
train_samples = scaler.transform(train_samples)
test_samples = scaler.transform(test_samples)

# uczenie
from sklearn.neighbors import KNeighborsClassifier
k = 3
classifier = KNeighborsClassifier(n_neighbors = k)
classifier.fit(train_samples, train_labels)


# 4. Klasyfikacja nieznanych próbek
predicted_labels = classifier.predict(test_samples)


# 5. Ocena klasyfikatora
# print(test_labels == predicted_labels)
quality = np.sum(test_labels == predicted_labels) / len(test_labels) * 100;
print('Spectral_rolloff: Ocena klasyfikatora: %.2f %%' % quality)

Spectral_rolloff: Ocena klasyfikatora: 4.85 %


In [11]:
# 1. Ekstrakcja cech z próbek
# Lista ekstraktorów cech: https://librosa.github.io/librosa/feature.html

# ścieżka z folderami z nagraniami - próbki:
train_audio_path = 'C:/Users/Aayli/Documents/Delete'
commands = os.listdir(train_audio_path);  


# wczytujemy każde nagranie, wykonujemy ekstrakcję i zapamiętujemy jej wynik
classes = []
class_size = 100 # użyjemy tylko po kilka danych
samples = []
labels = []
for command in commands:
    if command == 'testing_list.txt' or command == 'validation_list.txt' or command == '_background_noise_' or command == 'tmp':
        continue
    classes.append(command)
    sample_num = 0
    for file in os.listdir(train_audio_path + '/' + command + '/'):
        sample_num += 1
        y, sr = librosa.load(train_audio_path + '/' + command + '/' + file)
        mfcc_seq = librosa.feature.zero_crossing_rate(y)
        mean_mfcc_seq = []
        for feature in mfcc_seq:
            mean_mfcc_seq.append(np.mean(feature))
        samples.append(mean_mfcc_seq)
        labels.append(command)
        if sample_num == class_size:
            break

# warto podglądnąć, dla mniejszych n_mfcc, np. 5 co kryje mfcc_seq i mean_mfcc_seq

# zapiszmy bazę do późniejszej pracy
database = open('C:/Users/Aayli/Documents/Delete/tmp/database.pkl', 'wb')
pickle.dump([classes, samples, labels], database)
database.close()
# 2. Wybór danych treningowych i testowych

# Zautomatyzowany i losowy sposób to użycie:

# wczytanie danych
database = open('C:/Users/Aayli/Documents/Delete/tmp/database.pkl', 'rb')
classes, samples, labels = pickle.load(database)
database.close()

# wybierzmy 10 pierwszych próbek z każdej klasy jako uczące
# oraz kolejne 2 z każdej klasy jako testowe
# uwaga! train_size + test_size NIE może przekroczyć class_size (poprzedni punkt)
train_size = 12
test_size = 10

train_samples = []
train_labels = []
test_samples = []
test_labels = []
train_samples, test_samples, train_labels, test_labels = train_test_split(samples, labels, test_size=0.33, random_state=42)

# 3. Uczenie klasyfikatora danymi treningowymi

# Lista klasyfikatorów: https://stackabuse.com/overview-of-classification-methods-in-python-with-scikit-learn/
# Prosty tutorial: https://www.digitalocean.com/community/tutorials/how-to-build-a-machine-learning-classifier-in-python-with-scikit-learn

# skalowanie/normalizacja danych
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(train_samples)
train_samples = scaler.transform(train_samples)
test_samples = scaler.transform(test_samples)

# uczenie
from sklearn.neighbors import KNeighborsClassifier
k = 3
classifier = KNeighborsClassifier(n_neighbors = k)
classifier.fit(train_samples, train_labels)


# 4. Klasyfikacja nieznanych próbek
predicted_labels = classifier.predict(test_samples)


# 5. Ocena klasyfikatora
# print(test_labels == predicted_labels)
quality = np.sum(test_labels == predicted_labels) / len(test_labels) * 100;
print('Zero_crossing_rate: Ocena klasyfikatora: %.2f %%' % quality)

Zero_crossing_rate: Ocena klasyfikatora: 4.65 %


In [13]:
# 1. Ekstrakcja cech z próbek
# Lista ekstraktorów cech: https://librosa.github.io/librosa/feature.html

# ścieżka z folderami z nagraniami - próbki:
train_audio_path = 'C:/Users/Aayli/Documents/Delete'
commands = os.listdir(train_audio_path);  


# wczytujemy każde nagranie, wykonujemy ekstrakcję i zapamiętujemy jej wynik
classes = []
class_size = 100 # użyjemy tylko po kilka danych
samples = []
labels = []
for command in commands:
    if command == 'testing_list.txt' or command == 'validation_list.txt' or command == '_background_noise_' or command == 'tmp':
        continue
    classes.append(command)
    sample_num = 0
    for file in os.listdir(train_audio_path + '/' + command + '/'):
        sample_num += 1
        y, sr = librosa.load(train_audio_path + '/' + command + '/' + file)
        mfcc_seq = librosa.feature.rms(y=y)
        mean_mfcc_seq = []
        for feature in mfcc_seq:
            mean_mfcc_seq.append(np.mean(feature))
        samples.append(mean_mfcc_seq)
        labels.append(command)
        if sample_num == class_size:
            break

# warto podglądnąć, dla mniejszych n_mfcc, np. 5 co kryje mfcc_seq i mean_mfcc_seq

# zapiszmy bazę do późniejszej pracy
database = open('C:/Users/Aayli/Documents/Delete/tmp/database.pkl', 'wb')
pickle.dump([classes, samples, labels], database)
database.close()
# 2. Wybór danych treningowych i testowych

# Zautomatyzowany i losowy sposób to użycie:

# wczytanie danych
database = open('C:/Users/Aayli/Documents/Delete/tmp/database.pkl', 'rb')
classes, samples, labels = pickle.load(database)
database.close()

# wybierzmy 10 pierwszych próbek z każdej klasy jako uczące
# oraz kolejne 2 z każdej klasy jako testowe
# uwaga! train_size + test_size NIE może przekroczyć class_size (poprzedni punkt)
train_size = 12
test_size = 10

train_samples = []
train_labels = []
test_samples = []
test_labels = []
train_samples, test_samples, train_labels, test_labels = train_test_split(samples, labels, test_size=0.33, random_state=42)

# 3. Uczenie klasyfikatora danymi treningowymi

# Lista klasyfikatorów: https://stackabuse.com/overview-of-classification-methods-in-python-with-scikit-learn/
# Prosty tutorial: https://www.digitalocean.com/community/tutorials/how-to-build-a-machine-learning-classifier-in-python-with-scikit-learn

# skalowanie/normalizacja danych
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(train_samples)
train_samples = scaler.transform(train_samples)
test_samples = scaler.transform(test_samples)

# uczenie
from sklearn.neighbors import KNeighborsClassifier
k = 3
classifier = KNeighborsClassifier(n_neighbors = k)
classifier.fit(train_samples, train_labels)


# 4. Klasyfikacja nieznanych próbek
predicted_labels = classifier.predict(test_samples)


# 5. Ocena klasyfikatora
# print(test_labels == predicted_labels)
quality = np.sum(test_labels == predicted_labels) / len(test_labels) * 100;
print('RMS: Ocena klasyfikatora: %.2f %%' % quality)

RMS: Ocena klasyfikatora: 2.53 %


In [14]:
# 1. Ekstrakcja cech z próbek
# Lista ekstraktorów cech: https://librosa.github.io/librosa/feature.html

# ścieżka z folderami z nagraniami - próbki:
train_audio_path = 'C:/Users/Aayli/Documents/Delete'
commands = os.listdir(train_audio_path);  


# wczytujemy każde nagranie, wykonujemy ekstrakcję i zapamiętujemy jej wynik
classes = []
class_size = 100 # użyjemy tylko po kilka danych
samples = []
labels = []
for command in commands:
    if command == 'testing_list.txt' or command == 'validation_list.txt' or command == '_background_noise_' or command == 'tmp':
        continue
    classes.append(command)
    sample_num = 0
    for file in os.listdir(train_audio_path + '/' + command + '/'):
        sample_num += 1
        y, sr = librosa.load(train_audio_path + '/' + command + '/' + file)
        mfcc_seq = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        mean_mfcc_seq = []
        for feature in mfcc_seq:
            mean_mfcc_seq.append(np.mean(feature))
        samples.append(mean_mfcc_seq)
        labels.append(command)
        if sample_num == class_size:
            break

# warto podglądnąć, dla mniejszych n_mfcc, np. 5 co kryje mfcc_seq i mean_mfcc_seq

# zapiszmy bazę do późniejszej pracy
database = open('C:/Users/Aayli/Documents/Delete/tmp/database.pkl', 'wb')
pickle.dump([classes, samples, labels], database)
database.close()
# 2. Wybór danych treningowych i testowych

# Zautomatyzowany i losowy sposób to użycie:

# wczytanie danych
database = open('C:/Users/Aayli/Documents/Delete/tmp/database.pkl', 'rb')
classes, samples, labels = pickle.load(database)
database.close()

# wybierzmy 10 pierwszych próbek z każdej klasy jako uczące
# oraz kolejne 2 z każdej klasy jako testowe
# uwaga! train_size + test_size NIE może przekroczyć class_size (poprzedni punkt)
train_size = 12
test_size = 10

train_samples = []
train_labels = []
test_samples = []
test_labels = []
train_samples, test_samples, train_labels, test_labels = train_test_split(samples, labels, test_size=0.33, random_state=42)

# 3. Uczenie klasyfikatora danymi treningowymi

# Lista klasyfikatorów: https://stackabuse.com/overview-of-classification-methods-in-python-with-scikit-learn/
# Prosty tutorial: https://www.digitalocean.com/community/tutorials/how-to-build-a-machine-learning-classifier-in-python-with-scikit-learn

# skalowanie/normalizacja danych
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(train_samples)
train_samples = scaler.transform(train_samples)
test_samples = scaler.transform(test_samples)

# uczenie
from sklearn.neighbors import KNeighborsClassifier
k = 3
classifier = KNeighborsClassifier(n_neighbors = k)
classifier.fit(train_samples, train_labels)


# 4. Klasyfikacja nieznanych próbek
predicted_labels = classifier.predict(test_samples)


# 5. Ocena klasyfikatora
# print(test_labels == predicted_labels)
quality = np.sum(test_labels == predicted_labels) / len(test_labels) * 100;
print('Spectral_bandwidth: Ocena klasyfikatora: %.2f %%' % quality)

Spectral_bandwidth: Ocena klasyfikatora: 4.85 %
